In [10]:
from google.colab import drive
drive.mount('/content/gdrive') 


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
#libraries necessary for prediction
import pandas as pd 
import numpy as np

from sklearn.decomposition import PCA as sklearnPCA
from sklearn.naive_bayes import BernoulliNB

In [12]:
#train-test data is loaded to predict the output
def Dataforactivityprediction(training_dataset,testing_dataset):
    
    def convertDataframe(datainput):
        dataframe = pd.DataFrame(columns=range(100000))
        
        for z in range(len(datainput)):
            value = np.fromstring(datainput[z], dtype=int, sep=' ')
            bool_value = [0 for j in range(100000)]
            for col in value:
                bool_value[col-1] = 1
            
            dataframe.loc[z] = bool_value
            
        return dataframe
    
    with open('/content/gdrive/MyDrive/trainfile.txt') as f1:
        trainFile = f1.readlines()
    
    #the lines in test and train data is split as data and labels
    trainingdata_list = []
    traininglabels_list = []
    
    for Datainput in trainFile:
        traininglabels_list.append(Datainput[0])
        
        
        Datainput = Datainput.replace("0\t", "") #label (0/1) is erased
        Datainput = Datainput.replace("1\t", "") #new line is also added
        Datainput = Datainput.replace("\n", "")
        trainingdata_list.append(Datainput)
        traininglabels = np.asarray(traininglabels_list)
    training_datavalue = convertDataframe(trainingdata_list)
        
    with open('/content/gdrive/MyDrive/testfile.txt') as f2:
        testFile = f2.readlines()
    
    testing_datavalue = convertDataframe(testFile)
            
    return training_datavalue, testing_datavalue, traininglabels



In [13]:
def value_pca(traindata,testdata, s): #PCA is used to reduce dimensionality

    pcaresult = sklearnPCA(n_components = s)
    Estimated_PCAValue_trainData = pcaresult.fit_transform(training_datavalue)
    Estimated_PCAValue_testData = pcaresult.transform(testing_datavalue)
    return Estimated_PCAValue_trainData, Estimated_PCAValue_testData #reduced dimensionality of the data is projected 

#Classification to predict the output
def NaiveBayes(Estimated_PCAValue_trainData,Estimated_PCAValue_testData,  traininglabels ):

    NB = BernoulliNB()
    NB.fit(Estimated_PCAValue_trainData, traininglabels)

    Expected_predictions = []

    Expected_predictions = NB.predict(Estimated_PCAValue_testData)

    return Expected_predictions

In [15]:
#dataset loaded  is read and three distinct dataframe is obtained of training, testing data and train labels
training_datavalue, testing_datavalue, traininglabels = Dataforactivityprediction('trainfile.txt','testfile.txt')


In [17]:
#Dimensionality is reduced from 1000 to 100 as expected using PCA :( Principal Component Analysis ) 
Estimated_PCAValue_trainData, Estimated_PCAValue_testData = value_pca(training_datavalue, testing_datavalue, 500)

In [18]:
Expected_predictions = NaiveBayes(Estimated_PCAValue_trainData, Estimated_PCAValue_testData, traininglabels )

In [23]:
predictedop = open('/content/gdrive/MyDrive/outputF.txt', 'w')

predictedop.writelines( "%s\n" % prediction for prediction in Expected_predictions)

predictedop.close()